In [ ]:
'''
talib
'''

In [ ]:
import talib
import numpy as np

In [ ]:
high = np.array([1.0, 2.0, 6.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
low = np.array([1.0, 2.0, 5.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0])
close = np.array([1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
a = talib.ATR(high, low, close, 2)
a

In [ ]:
close = np.array([1000.0, 2.0, 1.0, 6.0, 6.0, 1.0, 6.0, 1.0, 1.0, 1.0, 1.0])
result = talib.RSI(close, 2)
result

In [ ]:
'''
tushare
'''

In [1]:
import tushare as ts
from datetime import datetime, timedelta

In [2]:
cons = ts.get_apis()

In [ ]:
df = ts.tick('rb1805', conn = cons, date = '2017-06-01', asset = 'X') 
#df = ts.tick('IF1801', conn = cons, date = '2017-06-22', asset = 'X') 
#df = ts.tick('600380', conn = cons, date = '2018-01-02')

In [ ]:
df.tocs

In [ ]:
for i, row in df.iterrows():
    print row['date'], '*'*2, row['vol']

In [ ]:
df = ts.bar('rb1805', conn = cons, freq = 'D', start_date = '2017-01-01', end_date = '', asset = 'X')

In [ ]:
df.sort_index(inplace = True)
df

In [ ]:
a = df.ix[1]
the = a['date']
dt = the.to_datetime()
print dt
if dt.time() >= datetime.time(21, 0):
    dt = dt - timedelta(1)

print dt

In [ ]:
for index, row in df.iterrows():
    print row['date'], ' '*2, row['vol']

In [ ]:
from datetime import datetime

In [ ]:
from collections import namedtuple
abc = namedtuple('xyz', ['x', 'y'])
the = abc(6,2)
the.x

In [ ]:
import sys
#sys.getdefaultencoding()
#reload(sys)
#sys.setdefaultencoding('UTF-8')
a = sys.getdefaultencoding()
type(a)

In [ ]:
if isinstance('a', str):
    print 1
else:
    print 2

In [ ]:
import pymongo 

In [ ]:
c = pymongo.MongoClient('localhot', 270)
if c:
    print c
    
else:
    print 'no'

In [ ]:
'''
datetime
'''

In [ ]:
from datetime import datetime, time

In [ ]:
a = datetime.now().time()
print type(a)

In [ ]:
a = False
if not a:
    print 'yes'

In [28]:
'''
json
'''
import json

In [29]:
dic = {'a':1,
       'b':2,
       'c':3}
arr = [dic]
dicJson = json.dumps(arr)
filePath = 'clone\\xyz.json'
f = open(filePath, 'w')
f.write(dicJson)
f.close()

In [30]:
f1 = open(filePath)
the = json.load(f1)
for dic in the:
    print dic['a']

1


In [24]:
import os

In [25]:
currentFolder = os.getcwd()
print currentFolder
newPath = os.path.join(currentFolder, '')

'C:\\Users\\Rachel\\Quant\\python-for-investment\\jupyter'

In [31]:
d = datetime.now().strftime('Y-m-d')
d

'Y-m-d'

In [42]:

a = datetime.strptime('2018-01-08', '%Y-%m-%d').date()
b = datetime.now().date()
print type(a)
print type(b)
print a
print b
if a == b:
    print 'yes'
else:
    print 'no'

<type 'datetime.date'>
<type 'datetime.date'>
2018-01-08
2018-01-08
yes


In [39]:
a

datetime.date(2018, 1, 8)

In [40]:
b

datetime.date(2018, 1, 8)

In [41]:
if datetime.now().strftime('%Y-%m-%d') == '2018-01-08':
    print 'yes'
else:
    print 'no'

yes
